In [ ]:
import os
import subprocess
import sys
from google.colab import drive
from concurrent.futures import ThreadPoolExecutor

# 1. Mount Drive
drive.mount('/content/drive', force_remount=True)

# 2. Install Tools (Using Nightly Build to fix Tubi errors)
!pip install -U git+https://github.com/yt-dlp/yt-dlp.git -q

# 3. Inputs
url = input("Paste Tubi Link: ")
save_path_base = "/content/drive/MyDrive/TubiShows"

# 4. Show Info & Get Show Name
print("\n--- Analyzing Show ---")
raw_title = subprocess.getoutput(f'yt-dlp --get-filename -o "%(playlist_title)s" --playlist-items 1 "{url}"').split('\n')[0]

# Handle cases where title extraction fails due to errors
if "ERROR" in raw_title or "Unable" in raw_title:
    print(f"\n⚠️ Could not auto-detect title. (Server response: {raw_title[:40]}...)")
    show_title = input("Please enter the Show Name manually: ")
else:
    show_title = raw_title if raw_title != "NA" else input("Show name not found. Enter name: ")

save_path = f"{save_path_base}/{show_title}"
if not os.path.exists(save_path): os.makedirs(save_path)

# 5. List Episodes
print("\n--- Fetching Episode List ---")
!yt-dlp --get-title --get-id --playlist-items 1:200 "{url}" > info.txt
with open('info.txt', 'r') as f:
    lines = f.readlines()

# Safety Check: Did we get episodes?
if len(lines) < 2:
    print("\n❌ ERROR: No episodes found! Tubi might be blocking the request or the link is invalid.")
    sys.exit()

titles, ids = lines[0::2], lines[1::2]

for i, t in enumerate(titles, 1): print(f"{i}. {t.strip()}")
selection = input(f"\nEnter range (e.g., 1-10) or single number (e.g., 5): ")

# Handle both ranges and single numbers
if '-' in selection:
    start, end = map(int, selection.split('-'))
else:
    start = int(selection)
    end = start

# Fix index out of bounds
if end > len(ids): end = len(ids)
if start < 1: start = 1

selected_ids = ids[start-1:end]
total_to_download = len(selected_ids)

if total_to_download == 0:
    print("❌ Selection invalid or empty.")
    sys.exit()

# 6. Progress Function
def download_item(index, video_id):
    current_num = index + 1
    # Using \r to keep the progress on one line
    print(f"\rDownloading {current_num}/{total_to_download}...", end="", flush=True)
    
    video_url = f"https://tubitv.com/movies/{video_id.strip()}"
    cmd = [
        "yt-dlp", "-q", "--no-progress", "-f", "bestvideo+bestaudio/best",
        "--merge-output-format", "mp4", "--no-write-subs",
        "-o", f"{save_path}/%(title)s.%(ext)s", video_url
    ]
    subprocess.run(cmd)
    print(f"\rUploading {current_num}/{total_to_download}...  ", end="", flush=True)

# 7. Execute Parallel (5 at a time)
print(f"\n🚀 Starting Batch Download for {show_title}...")
with ThreadPoolExecutor(max_workers=5) as executor:
    list(executor.map(lambda x: download_item(x[0], x[1]), enumerate(selected_ids)))

print(f"\n\n✅ ALL DONE! Check Drive: TubiShows/{show_title}")